In [1]:
# saves you having to use print as all exposed variables are printed in the cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# suppress warning message
import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.compute as pc

---------

In [4]:
from nn_rag import Knowledge
from nn_rag.components.commons import Commons